### This script is designed to get indexes of invisible beacon from the beacon file by comparing intervals if less than 60 seconds between
### sanity check is then to compare to metadata if for some session it fits. 
### next step is to be able to index into the position file and get behavioral state or outcome - comparing to straightness trajectories. 

In [37]:
root3 = '//10.153.170.3/storage2/fabian/data/raw/FS03/Event_files_FS03/'
root4 = '//10.153.170.3/storage2/fabian/data/raw/FS04/Event_files_FS04'
figures = 'C:/Users/Fabian/Desktop/Analysis/Round3_FS03_FS06/Figures/'
animal = '//10.153.170.3/storage2/fabian/data/raw/FS03/Event_files_FS03/'

### Individual session example. 

In [99]:
FS03_position= pd.read_csv(root3+'position_20210312-172823.txt',sep=" ", header=None)
#FS04_position = pd.read_csv(root4+'position 20200128-151826.txt',sep=" ", header=None)

FS03_beacon = pd.read_csv(root+'beacons_20210312-172823.txt',sep=" ", header=None)
#FS04_beacon = pd.read_csv(root+'beacons 20200128-151826.txt',sep=" ", header=None)

FS03_metadata = pd.read_csv(root+'metadata_20210312-172823.txt',sep=" ", header=None)
#FS04_metadata = pd.read_csv(root+'beacons 20200128-151826.txt',sep=" ", header=None)

In [123]:
FS03_beacon.head()
pd.options.display.float_format = '{:.5f}'.format
FS03_beacon.head()

,0,1,2,3,4,5
0,1615566511.42000,-0.40339,0.69336,0.30391,-0.17745,-0.52423
1,1615566575.04000,-0.47736,0.68880,0.28572,-0.17745,-0.52423
2,1615566591.52000,-0.48584,0.65964,0.33947,-0.17745,-0.52423
3,1615566654.67000,-0.43568,0.69867,0.32623,-0.17745,-0.52423
4,1615566660.58000,-0.43229,0.72260,0.29120,-0.17745,-0.52423


### diffs = np.diff(trajectory, axis=0)

dt = diffs[:, 0]
dx = np.sqrt(np.square(diffs[:, 1]) + np.square(diffs[:, 2]))


In [102]:
df = FS03_beacon[0].diff().to_frame()
df.head()

,0
0,NaN
1,63.62
2,16.48
3,63.15
4,5.91


In [108]:
invis=[]
#for ind in FS03_beacon.index:
df = FS03_beacon[0].diff().to_frame()
for ind in df.index:
    if df[0][ind]<60 and not (ind % 2 == 0):
        invis.append(ind)
        print(FS03_beacon[0][ind])

        
invis

1615566694.1
1615566894.46
1615567446.89
1615567680.72
1615568260.29


[5, 11, 27, 33, 47]

In [129]:
pos_index=[]
invis=[]
#for ind in FS03_beacon.index:
df = FS03_beacon[0].diff().to_frame()
for ind in df.index:
    if df[0][ind]<60 and not (ind % 2 == 0):
        invis.append(ind)
        print(FS03_beacon[0][ind])
        k = FS03_position.iloc[(FS03_position[0] - FS03_beacon[0][ind]).abs().argsort()[:1]]
        #print(list(k.index))
        print(FS03_position[0][list(k.index)])
        pos_index.append(int(''.join(map(str,list(k.index)))))
invis
pos_index

1615566694.1
11402   1615566694.10000
Name: 0, dtype: float64
1615566894.46
23433   1615566894.46000
Name: 0, dtype: float64
1615567446.89
56610   1615567446.90000
Name: 0, dtype: float64
1615567680.72
70657   1615567680.72000
Name: 0, dtype: float64
1615568260.29
105457   1615568260.30000
Name: 0, dtype: float64


[11402, 23433, 56610, 70657, 105457]

### This is almost correct - so now I can try to match position of animal with the time.

### Sanity check

In [39]:

result=pd.DataFrame()

for dirpath, dirnames, files in os.walk(root3, topdown=True):
    fullstring = dirpath
    for metadata in files:
        if fnmatch.fnmatch(metadata, 'metadata_*'):
            k=(animal+'/'+metadata)
            day = pd.read_csv(k,sep=" : ", header=None,engine='python')
            df=day.T
            df= df.rename(columns=df.iloc[0])
            df=df.drop(df.index[0])
            if int(df['Pellets'].values[0])>1:
                result = result.append(df, ignore_index=True,sort=False)
sorted_data = result.sort_values('Computer time was',)
sorted_data

,Recording started on,Computer time was,ITI_time,time_in_cylinder,movement_collection_time,animal_ID,background_color,circle,position_change,light_off,...,Speed,recording lenght,high pellets,high_time_in_cylinder,invisible_time,invisible_count,invisible_list,Animal jumped,Recording_started_motive,Recording_timestamp_motive
3,2020-12-17 17:12:09,1608221529.46,1.5,0.5,0.01,FS3,000,0.075,10,2,...,52.75555061359296,417.196999788,8,0.25,60,5,"[1, 5, 7, 13, 15]",-1,5731039.77242,0.0
4,2020-12-17 17:24:20,1608222260.01,1.5,2.5,0.01,FS3,000,0.075,10,2,...,38.55947434005755,1831.65300012,45,0.5,60,5,"[1, 7, 17, 35, 39]",7,5731770.29742,1831.80000008
8,2020-12-18 20:23:33,1608319413.97,1.5,2.5,0.01,FS3,000,0.075,10,2,...,42.783514388176535,1804.55100012,37,0.5,60,0,[],88,5828923.95576,1804.76666676
2,2020-12-20 17:46:18,1608482778.97,1.5,2.5,0.01,FS3,000,0.075,10,2,...,27.71136119439565,1818.45700002,32,0.5,60,0,[],3,5992283.64742,1818.61666642
0,2020-12-21 12:29:43,1608550183.31,1.5,2.5,0.01,FS3,000,0.075,10,2,...,25.686766856345457,1822.67599988,36,0.5,60,3,"[7, 15, 19]",4,6059687.93076,1822.80000008
20,2020-12-21 21:14:12,1608581652.63,1.5,2.5,0.01,FS3,000,0.075,10,2,...,38.49684244793969,1914.77699995,60,0.5,60,11,"[3, 7, 9, 33, 35, 37, 51, 53, 55, 57, 59]",31,6091157.19742,1914.90833337
1,2020-12-22 14:49:48,1608644988.1,1.5,2.5,0.01,FS3,000,0.075,10,2,...,42.69882141685956,1825.50100017,42,0.5,60,0,[],13,6154493.43909,1825.67500013
10,2020-12-22 21:49:12,1608670152.04,1.5,2.5,0.01,FS3,000,0.075,10,2,...,43.358106854653236,1817.04699993,44,0.5,60,1,[9],51,6179657.38076,1817.17500033
9,2020-12-23 12:51:51,1608724311.47,1.5,2.5,0.01,FS3,000,0.075,10,2,...,36.84846823940846,1851.10100007,40,0.5,60,3,"[29, 33, 35]",4,6233816.63076,1851.25833323
5,2020-12-23 20:55:56,1608753356.69,1.5,2.5,0.01,FS3,000,0.075,10,2,...,41.72752787695465,1864.98000002,44,0.5,60,2,"[15, 25]",30,6262861.83909,1865.11666662


In [30]:
invis =[]
for ind in sorted_data.index:
    l=sorted_data['invisible_list'].values[ind]
    print(l)
    if not l:
        for num in l: 
            print(num)
            invis.append(l[num])
invis

[]
[7, 15, 19]
[29, 33, 35]
[]
[1, 5, 7, 13, 15]
[13, 23, 39, 43]
[1, 7, 17, 35, 39]
[15]
[15, 25]
[3, 7, 9, 33, 35, 37, 51, 53, 55, 57, 59]
[13, 15, 17, 19]
[21, 23, 25, 27, 47, 51]
[]
[]
[3, 7]
[15]
[]
[3]
[29]
[5, 29]
[29, 39, 45, 55, 61, 69]
[5, 11, 33, 47]
[19, 21, 33]
[9, 13, 21, 31, 39]
[33]
[9]
[]
[3, 5, 7, 9, 11, 27, 49]
[41]
[13, 25, 37]
[13, 17, 19]
[9, 17, 19, 27, 29, 37, 45, 49]
[31, 35, 37, 39, 49]
[33, 37, 39]
[]
[]
[23, 25, 27, 29, 43]
[15, 17]
[25, 29]
[9, 25, 27, 41]
[31, 33, 35, 37, 39, 41, 47, 49, 53]
[3, 13, 23, 29, 33, 37, 39, 47, 49, 55]
[1, 3, 7, 11, 13, 15, 17, 19]
[5, 9, 13, 17, 19, 21]
[17, 19, 31, 33, 35, 37, 41, 43, 45, 49]


[]

### imports 

In [5]:
import math
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from numpy import median
from scipy.stats import ranksums

import numpy as np
import scipy.stats

import os
import fnmatch